First, we will get the Single cell studies database:

In [1]:
import pandas as pd
data = pd.read_csv('http://nxn.se/single-cell-studies/data.tsv', sep='\t')

In [5]:
data.head(3)

,Shorthand,DOI,Authors,Journal,Title,Date,bioRxiv DOI,Reported cells total,Organism,Tissue,...,Developmental stage,Number of reported cell types or clusters,Cell clustering,Pseudotime,RNA Velocity,PCA,tSNE,H5AD location,Isolation,BC --> Cell ID _OR_ BC --> Cluster ID
0,Cauli et al PNAS,10.1073/pnas.97.11.6144,"B. Cauli, J. T. Porter, K. Tsuzuki, B. Lambole...",Proceedings of the National Academy of Sciences,Classification of fusiform neocortical interne...,20020726,-,85,Rat,Brain,...,21-27 days,3.0,Yes,No,No,Yes,No,NaN,Patch-clamp,NaN
1,Malnic et al Cell,10.1016/S0092-8674(00)80581-4,"Bettina Malnic, Junzo Hirono, Takaaki Sato, Li...",Cell,Combinatorial Receptor Codes for Odors,20040410,-,18,Mouse,Brain,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tietjen et al Neuron,10.1016/S0896-6273(03)00229-0,"Ian Tietjen, Jason M. Rihel, Yanxiang Cao, Geo...",Neuron,Single-Cell Transcriptional Analysis of Neuron...,20040415,-,37,"Human, Mouse",Brain,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,"Manual, LCM",NaN


In [7]:
data.columns

Index(['Shorthand', 'DOI', 'Authors', 'Journal', 'Title', 'Date',
       'bioRxiv DOI', 'Reported cells total', 'Organism', 'Tissue',
       'Technique', 'Data location', 'Panel size', 'Measurement',
       'Cell source', 'Disease', 'Contrasts', 'Developmental stage',
       'Number of reported cell types or clusters', 'Cell clustering',
       'Pseudotime', 'RNA Velocity', 'PCA', 'tSNE', 'H5AD location',
       'Isolation', 'BC --> Cell ID _OR_ BC --> Cluster ID'],
      dtype='object')

The idea is to reconcile interesting information to Wikidata. The following columns are considered interesting:


In [8]:
cols_of_interest = ["DOI", "Organism", "Tissue", "Technique", "Data location" ]

The idea is to reconcile interesting information to Wikidata. The following columns are considered interesting:


In [9]:
data = data[cols_of_interest]

In [10]:
data

,DOI,Organism,Tissue,Technique,Data location
0,10.1073/pnas.97.11.6144,Rat,Brain,sc-RT-mPCR,NaN
1,10.1016/S0092-8674(00)80581-4,Mouse,Brain,PCR,NaN
2,10.1016/S0896-6273(03)00229-0,"Human, Mouse",Brain,PCR,NaN
3,10.1093/cercor/bhj081,Rat,Brain,sc-RT-mPCR,NaN
4,10.1093/nar/gkl050,Mouse,ICM,aRNA amplification,GSE4309
...,...,...,...,...,...
1214,10.1186/s13059-021-02267-5,Human,Culture,Chromium,GSE142392
1215,10.1038/s42003-020-01625-6,Human,"Blood, Tumor",Chromium,GSE121638
1216,10.7554/eLife.62586,NaN,NaN,NaN,NaN
1217,10.3390/biom11020177,Human,Culture,NaN,NaN


In [20]:

dois = data["DOI"]

dois = ['"' + doi + '"' for doi in dois]
doi_string = " ".join(dois[0:100])

query = """
SELECT ?normalized_doi ?item  ?itemLabel
WHERE {
  {
    SELECT ?item ?normalized_doi WHERE {
      VALUES ?doi {
      
      """ + doi_string +"""
        
      }
      BIND(UCASE(?doi) AS ?normalized_doi)
      ?item wdt:P356 ?normalized_doi.
    }
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

In [21]:
doi_string

'"10.1073/pnas.97.11.6144" "10.1016/S0092-8674(00)80581-4" "10.1016/S0896-6273(03)00229-0" "10.1093/cercor/bhj081" "10.1093/nar/gkl050" "10.1093/cercor/bhl104" "10.1016/j.neures.2007.12.011" "10.1186/1471-2164-9-268" "10.1523/JNEUROSCI.0058-09.2009" "10.1038/NMETH.1315" "10.1073/pnas.0902161106" "10.1016/j.devcel.2010.02.012" "10.1016/j.stem.2010.03.015" "10.1101/gr.110882.110" "10.1371/journal.pone.0021208" "10.1038/nbt.2282" "10.1002/dvdy.23850" "10.1016/j.celrep.2012.08.003" "10.1016/j.cell.2012.08.023" "10.1038/ncb2709" "10.1186/gb-2013-14-4-r31" "10.1038/nature12172" "10.1038/ncb2798" "10.1038/nature12364" "10.1038/nsmb.2660" "10.1038/nmeth.2645" "10.1038/nmeth.2639" "10.1038/nmeth.2694" "10.1073/pnas.1319700110" "10.1101/gr.161034.113" "10.1038/nmeth.2772" "10.1126/science.1245316" "10.1126/science.1247651" "10.1101/003236" "10.1038/nbt.2859" "10.1038/nature13173" "10.1016/j.cell.2014.03.036" "10.1073/pnas.1402030111" "10.1016/j.celrep.2014.04.011" "10.1038/nature13437" "10.1126/

In [22]:
from wikidata2df import wikidata2df

dois = wikidata2df(query)

In [23]:
dois

,item,itemLabel,normalized_doi
0,Q30490498,Classification of NPY-expressing neocortical i...,10.1523/JNEUROSCI.0058-09.2009
1,Q30921133,Single-cell transcriptional analysis of neuron...,10.1016/S0896-6273(03)00229-0
2,Q28240611,mRNA-Seq whole-transcriptome analysis of a sin...,10.1038/NMETH.1315
3,Q33340138,Modelling and measuring single cell RNA expres...,10.1186/1471-2164-9-268
4,Q29616773,Combinatorial receptor codes for odors,10.1016/S0092-8674(00)80581-4
...,...,...,...
94,Q34494714,RNA-Seq of single prostate CTCs implicates non...,10.1126/SCIENCE.AAB0917
95,Q34674303,Single-cell transcriptomics reveals receptor t...,10.1126/SCIENCE.AAD2456
96,Q36191159,Intrinsic Age-Dependent Changes and Cell-Cell ...,10.1016/J.DEVCEL.2015.09.009
97,Q41861103,The dynamics and regulators of cell fate decis...,10.1038/NBT.2859
